In [ ]:
import tkinter as tk
from tkinter import filedialog, Text
import os

# Add accepted file formats here with a 2-tuple entry
accepted_file_formats = [("text file", "*.txt"),
                         ("executables","*.exe"),
                         ("all files","*.*")]
selected_filepaths = []
selected_filepaths2 = []

class FileSelectorGUI(tk.Tk):
    def __init__(self, *args, **kwargs):
        super().__init__()
        
        # Attributes 
        self.title_name=self._get_arg(0, "title_name", args, kwargs)
        self.min_size=self._get_arg(1, "min_size", args, kwargs)
        self.accepted_file_formats=self._get_arg(2, "accepted_file_formats", args, kwargs)
        self.selected_filepaths=self._get_arg(3, "selected_filepaths", args, kwargs)
        
        # Root size config
        self.minsize(self.min_size[0], self.min_size[1])
        self.title(self.title_name)
        
        # Widgets
        self.canvas = tk.Canvas(self, bg="#ADD8E6")
        self.canvas.place(relwidth=1, relheight=1, relx=0.5, rely=0.5, anchor="center")

        self.frame = tk.Frame(self, bg="white")
        self.frame.place(relwidth=0.8, relheight=0.8, relx=0.1, rely=0.1)

        # Scrollbar only works on canvas
        # Thus, we create a list_frame on a list_canvas on a frame
        self.list_canvas = tk.Canvas(self.frame)
        self.list_frame = tk.Frame(self.list_canvas)
        self.scrollbar = tk.Scrollbar(self.frame, orient="vertical", command=self.list_canvas.yview)

        self.list_canvas.configure(yscrollcommand=self.scrollbar.set)
        self.scrollbar.pack(side="right", fill="y")
        self.list_canvas.pack(side="left", fill="both", expand=True)
        self.list_canvas.create_window((0,0), window=self.list_frame, anchor='nw')

        self.list_frame.bind("<Configure>", lambda event, canvas=self.list_canvas: self._onFrameConfigure(self.canvas))

        # Buttons
        self.browse_btn = tk.Button(self, text="Browse", padx=10, pady=5, 
                             fg="white", bg="black",
                            command=self._open_browser)
        self.browse_btn.place(relx=0.40, rely=1, anchor="s")

        self.confirm_btn = tk.Button(self, text="Confirm", padx=10, pady=5,
                                      fg="white", bg="black",
                                      command=self._confirm_selection)
        self.confirm_btn.place(relx=0.60, rely=1, anchor="s")
    
    def show_GUI(self):
        self.mainloop()
        
    def _get_arg(self, index, kw, args, kwargs):
        return args[index] if (kw not in kwargs) else kwargs.get(kw)

    def _onFrameConfigure(self, canvas):
        # Reset the scroll region to encompass the inner frame
        self.canvas.configure(scrollregion=self.canvas.bbox("all"))
    
    def _clear_list(self):
        for widget in self.list_frame.winfo_children():
            if isinstance(widget, tk.Frame):
                widget.destroy()

    def _build_list(self):
        for index, filepath in enumerate(self.selected_filepaths):
            line_frame = tk.Frame(self.list_frame, bg="gray")
            line_frame.grid(sticky="w")
            label = tk.Label(line_frame, text=filepath, bg="gray", padx=5, pady=5)
            label.pack(side="left")
            rm_btn = tk.Button(line_frame, text="X", bg="red", fg="white")
            rm_btn.pack(side="right")
            rm_btn.bind("<Button-1>", lambda event, index=index: self._remove_filepath(event, index))
        
    def _confirm_selection(self):
            print("selected list of filepath(s) {}".format(self.selected_filepaths))
            self.destroy()

    def _remove_filepath(self, event, index):
        #handling_widget = event.widget
        #line_frame = handling_widget.master
        #line_frame.destroy()
        print("removed {}".format(self.selected_filepaths[index]))
        del self.selected_filepaths[index]
        self._clear_list()
        self._build_list()

    def _open_browser(self):
        self._clear_list()

        filepath = filedialog.askopenfilename(initialdir="/", title="Select File",
                                             filetypes=(accepted_file_formats))
        if (filepath != ""):
            self.selected_filepaths.append(filepath)
            print("selected {}".format(filepath))
        else:
            print("no file selected")

        self._build_list()
        
instance = FileSelectorGUI("Datasets 1", (400,400), accepted_file_formats, selected_filepaths)
instance2 = FileSelectorGUI(accepted_file_formats=accepted_file_formats, 
                            selected_filepaths=selected_filepaths2,
                            title_name="Datasets 2", 
                            min_size=(800,800))
instance.show_GUI()
instance2.show_GUI()